## Steps to build the Loan Eligibility model

1. Loading the dataset
2. Pre-processing the dataset
3. Building the Loan Prediction model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. Loading the dataset

In [ ]:
# importing libraries
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')



In [ ]:

# loading the dataset
train = pd.read_csv( r'/content/drive/My Drive/loan_prediction/loan_data.csv')
train.head()


## 2. Pre-processing the dataset

In [ ]:
# converting categories into numbers
train['Gender']= train['Gender'].map({'Male':0, 'Female':1})
train['Married']= train['Married'].map({'No':0, 'Yes':1})
train['Loan_Status']= train['Loan_Status'].map({'N':0, 'Y':1})

In [ ]:
# separating dependent and independent variables
X = train[['Gender', 'Married', 'ApplicantIncome', 'LoanAmount']]
y = train.Loan_Status

## 3. Building the Loan Prediction model

In [ ]:
# importing machine learning model
from sklearn.linear_model import LogisticRegression

In [ ]:
# training the logistic regression model
model = LogisticRegression() 
model.fit(X, y)

In [ ]:
# saving the model 
import pickle 
pickle_out = open("/content/drive/My Drive/loan_prediction/classifier.pkl", mode = "wb") 
pickle.dump(model, pickle_out) 
pickle_out.close()

## Deploying the machine learning model using streamlit

1. Building the Frontend of the application
2. Loading and Pre-processing the data
3. Building the Machine Learning model to automate Loan Eligibility
4. Deploying the application

## 1. Building the Frontend of the application

1.1 Install Required Libraries<br>
1.2 Creating the Frontend of the app using Streamlit

### 1.1 Install Required Libraries##

In [ ]:
# installing pyngrok
!pip install -q pyngrok

In [ ]:
# installing streamlit
!pip install -q streamlit

### 1.2. Creating the frontend of the app using streamlit

In [ ]:
# creating the script
%%writefile app.py

# importing required libraries
import pickle
import streamlit as st

# loading the trained model
pickle_in = open('/content/drive/My Drive/loan_prediction/classifier.pkl', 'rb') 
classifier = pickle.load(pickle_in)

# this is the main function in which we define our app  
def main():       
    # header of the page 
    html_temp = """ 
    <div style ="background-color:yellow;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Check your Loan Eligibility</h1> 
    </div> 
    """
    st.markdown(html_temp, unsafe_allow_html = True) 

    # following lines create boxes in which user can enter data required to make prediction 
    Gender = st.selectbox('Gender',("Male","Female","Other"))
    Married = st.selectbox('Marital Status',("Unmarried","Married","Other")) 
    ApplicantIncome = st.number_input("Monthly Income in Kenya Shillings") 
    LoanAmount = st.number_input("Loan Amount in Kenya Shillings")
    result =""
      
    # when 'Check' is clicked, make the prediction and store it 
    if st.button("Check"): 
        result = prediction(Gender, Married, ApplicantIncome, LoanAmount) 
        st.success('Your loan is {}'.format(result))
 
# defining the function which will make the prediction using the data which the user inputs 
def prediction(Gender, Married, ApplicantIncome, LoanAmount): 

    # 2. Loading and Pre-processing the data 

    if Gender == "Male":
        Gender = 0
    else:
        Gender = 1

    if Married == "Married":
        Married = 1
    else:
        Married = 0

    

    prediction = classifier.predict( 
        [[Gender, Married, ApplicantIncome, LoanAmount]])
     
    if prediction == 0:
        pred = 'Rejected'
    else:
        pred = 'Approved'
    return pred
     
if __name__=='__main__': 
    main()

## 4. Deploying the application

In [ ]:
# running the app
!streamlit run app.py &>/dev/null&

In [ ]:
# making the locally-hosted web application to be publicly accessible
from pyngrok import ngrok

public_url = ngrok.connect('8501')
public_url